## Importing the necessary libraries for the project execution

In [38]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Importing the Requests package to get the Wikipedia page 

In [39]:
import requests

### Reading the Wikipedia Page and return the html to the variable 'page'

In [40]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

### Importing the Beautifulsoup package and Parse the html in the 'page' variable, and store it in Beautiful Soup format

In [41]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

### Use function “prettify” to look at nested structure of HTML page

In [42]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":857830462,"wgRevisionId":857830462,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

### Returns the content which is enclosed within the tags

In [43]:
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [44]:
soup.a

<a id="top"></a>

In [45]:
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [46]:
soup.find_all('a')

[<a id="top"></a>,
 <a class="mw-jump-link" href="#mw-head">Jump to navigation</a>,
 <a class="mw-jump-link" href="#p-search">Jump to search</a>,
 <a href="/wiki/Postal_codes_in_Canada" title="Postal codes in Canada">postal codes in Canada</a>,
 <a href="/wiki/Toronto" title="Toronto">Toronto</a>,
 <a href="/wiki/Ontario" title="Ontario">Ontario</a>,
 <a href="/wiki/Canada_Post" title="Canada Post">Canada Post</a>,
 <a href="#cite_note-1">[1]</a>,
 <a class="mw-redirect" href="/wiki/Smartphones" title="Smartphones">smartphones</a>,
 <a href="/wiki/IPhone" title="IPhone">iPhone</a>,
 <a href="/wiki/BlackBerry" title="BlackBerry">BlackBerry</a>,
 <a href="#cite_note-2">[2]</a>,
 <a href="/wiki/CD-ROM" title="CD-ROM">CD-ROMs</a>,
 <a href="/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;action=edit&amp;section=1" title="Edit section: Toronto]] - 103 Canadian postal code#Forward sortation areas|FSAs]]">edit</a>,
 <a href="/wiki/North_York" title="North York">North York</a>,
 <a hr

### Find the right table: As we are seeking a table to extract information about state capitals, we should identify the right table first. Let’s write the command to extract information within all table tags.

In [47]:
all_tables=soup.find_all('table')
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>
 <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Pa

#### Now to identify the right table, we will use attribute “class” of table and use it to filter the right table. In chrome, you can check the class name by right click on the required table of web page –> Inspect element –> Copy the class name OR go through the output of above command find the class name of right table.

Extract the information to DataFrame: Here, we need to iterate through each row (tr) and then assign each element of tr (td) to a variable and append it to a list. 

In [48]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

#### Now to access value of each element, we will use “find(text=True)” option with each element. 

In [49]:
A=[]
B=[]
C=[]

for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

len(cells)
A

['M1A',
 'M2A',
 'M3A',
 'M4A',
 'M5A',
 'M5A',
 'M6A',
 'M6A',
 'M7A',
 'M8A',
 'M9A',
 'M1B',
 'M1B',
 'M2B',
 'M3B',
 'M4B',
 'M4B',
 'M5B',
 'M5B',
 'M6B',
 'M7B',
 'M8B',
 'M9B',
 'M9B',
 'M9B',
 'M9B',
 'M9B',
 'M1C',
 'M1C',
 'M1C',
 'M2C',
 'M3C',
 'M3C',
 'M4C',
 'M5C',
 'M6C',
 'M7C',
 'M8C',
 'M9C',
 'M9C',
 'M9C',
 'M9C',
 'M1E',
 'M1E',
 'M1E',
 'M2E',
 'M3E',
 'M4E',
 'M5E',
 'M6E',
 'M7E',
 'M8E',
 'M9E',
 'M1G',
 'M2G',
 'M3G',
 'M4G',
 'M5G',
 'M6G',
 'M7G',
 'M8G',
 'M9G',
 'M1H',
 'M2H',
 'M3H',
 'M3H',
 'M3H',
 'M4H',
 'M5H',
 'M5H',
 'M5H',
 'M6H',
 'M6H',
 'M7H',
 'M8H',
 'M9H',
 'M1J',
 'M2J',
 'M2J',
 'M2J',
 'M3J',
 'M3J',
 'M4J',
 'M5J',
 'M5J',
 'M5J',
 'M6J',
 'M6J',
 'M7J',
 'M8J',
 'M9J',
 'M1K',
 'M1K',
 'M1K',
 'M2K',
 'M3K',
 'M3K',
 'M4K',
 'M4K',
 'M5K',
 'M5K',
 'M6K',
 'M6K',
 'M6K',
 'M7K',
 'M8K',
 'M9K',
 'M1L',
 'M1L',
 'M1L',
 'M2L',
 'M2L',
 'M3L',
 'M4L',
 'M4L',
 'M5L',
 'M5L',
 'M6L',
 'M6L',
 'M6L',
 'M7L',
 'M8L',
 'M9L',
 'M1M',
 'M1M',


### Converting the lists to one dataframe

In [50]:
#import pandas to convert list to data frame
import pandas as pd
df=pd.DataFrame(A,columns=['Postal_code'])
df['Borough']=B
df['Neighborhood']=C

df

,Postal_code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


#### Removing the rows with incomplete data

In [51]:
df = df[df.Borough !="Not assigned"]
df = df[df.Neighborhood !="Not assigned"]
df

,Postal_code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


#### Resetting the Index

In [52]:
df.reset_index(drop=True)

,Postal_code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [53]:
df.shape

(212, 3)

## Below step joins values in Neighborhood column based on postalCode and Borough

In [54]:
df2=df.groupby(['Postal_code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df2

,Postal_code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood]]\n,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff,Cliffside West\n"


In [55]:
df2.shape

(103, 3)

In [56]:
postal = df.Postal_code.tolist()
len(postal)

212

### Creating 2 functions to get Latitude and Longitude
I'm using "Geolocator" tool insted of geocoder which was given to us for sample

I'm using Neighboorhod and Borough to get the latitude and longitude.

In [57]:
# Function for getting Latitude
def lat(elem):
    n,b=elem
    n1=list(n.split(","))
    a='{},{}'.format(n1[0],b)
    geolocator = Nominatim()
    location = geolocator.geocode(a)
    if location == None:
        location = geolocator.geocode(b)
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude = location.latitude
        longitude = location.longitude
    return latitude

In [58]:
# Function for getting Longitude
def lng(elem):
    n,b=elem
    n1=list(n.split(","))
    a='{},{}'.format(n1[0],b)
    #print(a)
    geolocator = Nominatim()
    location = geolocator.geocode(a)
    if location == None:
        location = geolocator.geocode(b)
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude = location.latitude
        longitude = location.longitude
    return longitude

In [59]:
df2['latitude'] = df2[['Neighborhood','Borough']].apply(lat,axis=1)

C:\Users\IBM_ADMIN\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [60]:
df2['longitude'] = df2[['Neighborhood','Borough']].apply(lng,axis=1)

C:\Users\IBM_ADMIN\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [62]:
df2.head()

,Postal_code,Borough,Neighborhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.804930,-79.165837
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.790117,-79.173334
2,M1E,Scarborough,"Guildwood]]\n,Morningside,West Hill",43.754899,-79.197776
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,43.756467,-79.226692


## Third part of the Question starts from Here

In [63]:
neighborhoods =df2

In [64]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        len(neighborhoods['Neighborhood'].unique())
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto.

In [65]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

C:\Users\IBM_ADMIN\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


### Creating a map of Toronto with neighborhoods superimposed on top

In [66]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto


### Considering North York as my sample cluster

In [67]:
North_York_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()


,Postal_code,Borough,Neighborhood,latitude,longitude
0,M2H,North York,Hillcrest Village,43.799664,-79.365019
1,M2J,North York,"Fairview\n,Henry Farm,Oriole\n",41.043430,-73.797355
2,M2K,North York,Bayview Village,43.769197,-79.376662
3,M2L,North York,"Silver Hills,York Mills",43.761222,-79.368382
4,M2M,North York,"Newtonbrook,Willowdale",43.793886,-79.425679


#### Get Latitude and Longitude for Etobicoke

In [68]:
address = 'North York, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

C:\Users\IBM_ADMIN\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of North York are 43.7709163, -79.4124102.


#### Creating a map of North York with neighborhoods superimposed on top.

In [69]:
# create map of New York using latitude and longitude values
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, borough, neighborhood in zip(North_York_data['latitude'], North_York_data['longitude'], North_York_data['Borough'], North_York_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_North_York)  
    
map_North_York

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them
#### Defining Foursquare Credentials and Version

In [70]:
CLIENT_ID = 'LSTPSCV1WUB2U1MDWUFASOWYY3VXINOS5PHALJ5QYE2IUICP' # your Foursquare ID
CLIENT_SECRET = 'O3YFLC4P5VOSTTOGRRUCH3FTBT0HYOJWGQUSQH0SM00D432W' # your Foursquare Secret
VERSION = '20180924' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LSTPSCV1WUB2U1MDWUFASOWYY3VXINOS5PHALJ5QYE2IUICP
CLIENT_SECRET:O3YFLC4P5VOSTTOGRRUCH3FTBT0HYOJWGQUSQH0SM00D432W


#### Here I'm taking Hillcrest Village for exploring

In [72]:
North_York_data.loc[0, 'Neighborhood']

'Hillcrest Village'

### Get the neighborhood's latitude and longitude values.

In [73]:
neighborhood_latitude = North_York_data.loc[9, 'latitude'] # neighborhood latitude value
neighborhood_longitude = North_York_data.loc[9, 'longitude'] # neighborhood longitude value

neighborhood_name = North_York_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.737178, -79.3434514.


#### Getting top 50 venues that are in Hillcrest Village within a radius of 700 meters.¶

In [74]:
#Limiting to 50 to save number of calls

LIMIT = 50 # limit of number of venues returned by Foursquare API

radius = 700 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LSTPSCV1WUB2U1MDWUFASOWYY3VXINOS5PHALJ5QYE2IUICP&client_secret=O3YFLC4P5VOSTTOGRRUCH3FTBT0HYOJWGQUSQH0SM00D432W&v=20180924&ll=43.737178,-79.3434514&radius=700&limit=50'

In [75]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ba89cf71ed21976f7e9e8ac'},
 'response': {'headerLocation': 'Sunnybrook - York Mills',
  'headerFullLocation': 'Sunnybrook - York Mills, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 34,
  'suggestedBounds': {'ne': {'lat': 43.74347800630001,
    'lng': -79.33474817403375},
   'sw': {'lat': 43.73087799369999, 'lng': -79.35215462596626}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc795d215a7ef3b97b279da',
       'name': 'CF Shops at Don Mills',
       'location': {'address': '1090 Don Mills Rd.',
        'crossStreet': 'at Lawrence Ave E',
        'lat': 43.73501112909844,
        'lng': -79.34478724407985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.73501112909844,
          '

#### From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [76]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [77]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,CF Shops at Don Mills,Shopping Mall,43.735011,-79.344787
1,Paramount Fine Foods,Middle Eastern Restaurant,43.736547,-79.343473
2,Chipotle Mexican Grill,Mexican Restaurant,43.735010,-79.343596
3,Aroma Espresso Bar,Café,43.734400,-79.345204
4,JOEY Don Mills,American Restaurant,43.733990,-79.344734


In [78]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


# 2. Explore Neighborhoods in North York

### Creating a function to repeat the same process to all the neighborhoods in North York

In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Getting venues in North York

In [80]:
North_York_venues = getNearbyVenues(names=North_York_data['Neighborhood'],
                                   latitudes=North_York_data['latitude'],
                                   longitudes=North_York_data['longitude']
                                  )

Hillcrest Village
Fairview
,Henry Farm,Oriole

Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South

York Mills West

Willowdale West
Parkwoods
Don Mills North

Flemingdon Park,Don Mills South

Bathurst Manor,Downsview North
,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East

Downsview West
Downsview Central

Downsview Northwest

Victoria Village
Bedford Park,Lawrence Manor East

Lawrence Heights,Lawrence Manor
Glencairn
Maple Leaf Park,North Park
,Upwood Park

Humber Summit
Emery,Humberlea


In [81]:
print(North_York_venues.shape)
North_York_venues.head()

(429, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.799664,-79.365019,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,43.798391,-79.369187,Korean Restaurant
1,Hillcrest Village,43.799664,-79.365019,Galati,43.797831,-79.369410,Grocery Store
2,Hillcrest Village,43.799664,-79.365019,Cummer Park,43.799564,-79.371175,Park
3,Hillcrest Village,43.799664,-79.365019,Tim Hortons,43.798945,-79.369644,Coffee Shop
4,Hillcrest Village,43.799664,-79.365019,TD Canada Trust,43.798466,-79.368832,Bank


#### Checking how many venues were returned for each neighborhood

In [82]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North\n,Wilson Heights",4,4,4,4,4,4
Bayview Village,15,15,15,15,15,15
"Bedford Park,Lawrence Manor East\n",2,2,2,2,2,2
"CFB Toronto,Downsview East\n",50,50,50,50,50,50
Don Mills North\n,34,34,34,34,34,34
Downsview Central\n,9,9,9,9,9,9
Downsview Northwest\n,9,9,9,9,9,9
Downsview West,9,9,9,9,9,9
"Emery,Humberlea",7,7,7,7,7,7


In [83]:
print('There are {} uniques categories.'.format(len(North_York_venues['Venue Category'].unique())))

There are 117 uniques categories.


# 3. Analyze Each Neighborhood
### One Hot encoding

In [84]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.tail()

,Neighborhood,American Restaurant,Arts & Crafts Store,Auto Dealership,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Store,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Empanada Restaurant,Fast Food Restaurant,Fish Market,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hotel,Ice Cream Shop,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Laundry Service,Liquor Store,Locksmith,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Office,Other Repair Shop,Outdoor Supply Store,Pakistani Restaurant,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wings Joint,Women's Store
424,"Emery,Humberlea",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
425,"Emery,Humberlea",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
426,"Emery,Humberlea",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
427,"Emery,Humberlea",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
428,"Emery,Humberlea",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [85]:
North_York_onehot.shape

(429, 118)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [86]:
North_York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_grouped

,Neighborhood,American Restaurant,Arts & Crafts Store,Auto Dealership,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Store,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Empanada Restaurant,Fast Food Restaurant,Fish Market,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hotel,Ice Cream Shop,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Laundry Service,Liquor Store,Locksmith,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Office,Other Repair Shop,Outdoor Supply Store,Pakistani Restaurant,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North\n,Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.066667,0.066667,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Bedford Park,Lawrence Manor East\n",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.50,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000

In [87]:
North_York_grouped.shape

(24, 118)

#### Printing each neighborhood along with the top 5 most common venues

In [90]:
num_top_venues = 5

for hood in North_York_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = North_York_grouped[North_York_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North
,Wilson Heights----
                 venue  freq
0    Convenience Store  0.25
1           Playground  0.25
2       Baseball Field  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Bayview Village----
                 venue  freq
0  Sporting Goods Shop  0.07
1          Pizza Place  0.07
2        Grocery Store  0.07
3   Persian Restaurant  0.07
4       Clothing Store  0.07


----Bedford Park,Lawrence Manor East
----
                        venue  freq
0  Construction & Landscaping   0.5
1                   Locksmith   0.5
2         American Restaurant   0.0
3                      Office   0.0
4                        Pool   0.0


----CFB Toronto,Downsview East
----
                 venue  freq
0    Korean Restaurant  0.08
1     Ramen Restaurant  0.08
2           Restaurant  0.06
3          Coffee Shop  0.06
4  Japanese Restaurant  0.04


----Don Mills North
----
                 venue  freq
0  American Restaurant  0.09
1          Cof

#### Function to sort the venues in descending order.

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating a new dataframe and displaying the top 10 venues for each neighborhood.

In [93]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = North_York_grouped['Neighborhood']

for ind in np.arange(North_York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North\n,Wilson Heights",Convenience Store,Playground,Park,Baseball Field,Food Court,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop
1,Bayview Village,Hardware Store,Breakfast Spot,Persian Restaurant,Coffee Shop,Clothing Store,Outdoor Supply Store,Sandwich Place,Fast Food Restaurant,Fish Market,Sporting Goods Shop
2,"Bedford Park,Lawrence Manor East\n",Construction & Landscaping,Locksmith,Women's Store,Food Truck,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store
3,"CFB Toronto,Downsview East\n",Ramen Restaurant,Korean Restaurant,Restaurant,Coffee Shop,Japanese Restaurant,Pizza Place,Sandwich Place,Café,Ice Cream Shop,Pet Store
4,Don Mills North\n,American Restaurant,Clothing Store,Coffee Shop,Gourmet Shop,Pizza Place,Pet Store,Movie Theater,Middle Eastern Restaurant,Mexican Restaurant,Liquor Store
5,Downsview Central\n,Coffee Shop,Bus Station,Gas Station,French Restaurant,Gym Pool,Outdoor Supply Store,Furniture / Home Store,Women's Store,Fish Market,Fast Food Restaurant
6,Downsview Northwest\n,Coffee Shop,Bus Station,Gas Station,French Restaurant,Gym Pool,Outdoor Supply Store,Furniture / Home Store,Women's Store,Fish Market,Fast Food Restaurant
7,Downsview West,Coffee Shop,Bus Station,Gas Station,French Restaurant,Gym Pool,Outdoor Supply Store,Furniture / Home Store,Women's Store,Fish Market,Fast Food Restaurant
8,"Emery,Humberlea",Fast Food Restaurant,Coffee Shop,Sandwich Place,Vietnamese Restaurant,Caribbean Restaurant,Tea Room,Fish Market,Empanada Restaurant,Electronics Store,Donut Shop
9,"Fairview\n,Henry Farm,Oriole\n",Women's Store,Furniture / Home Store,Shopping Mall,Shoe Store,Pizza Place,Mobile Phone Shop,Pet Store,Discount Store,Donut Shop,Rental Car Location


# 4. Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [94]:
# set number of clusters
kclusters = 5

North_York_grouped_clustering = North_York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 3, 2, 2, 1, 1, 1, 2, 2])

#### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [95]:
North_York_merged = North_York_data

# add clustering labels
North_York_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

North_York_merged.head() # check the last columns!

,Postal_code,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.799664,-79.365019,0,Ice Cream Shop,Korean Restaurant,Coffee Shop,Park,Recreation Center,Diner,Sandwich Place,Shopping Mall,Pharmacy,Pizza Place
1,M2J,North York,"Fairview\n,Henry Farm,Oriole\n",41.043430,-73.797355,2,Women's Store,Furniture / Home Store,Shopping Mall,Shoe Store,Pizza Place,Mobile Phone Shop,Pet Store,Discount Store,Donut Shop,Rental Car Location
2,M2K,North York,Bayview Village,43.769197,-79.376662,3,Hardware Store,Breakfast Spot,Persian Restaurant,Coffee Shop,Clothing Store,Outdoor Supply Store,Sandwich Place,Fast Food Restaurant,Fish Market,Sporting Goods Shop
3,M2L,North York,"Silver Hills,York Mills",43.761222,-79.368382,2,Middle Eastern Restaurant,Park,Tennis Court,Women's Store,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store
4,M2M,North York,"Newtonbrook,Willowdale",43.793886,-79.425679,2,Korean Restaurant,Coffee Shop,Vietnamese Restaurant,Middle Eastern Restaurant,American Restaurant,Park,Café,Cantonese Restaurant,Gym,Pizza Place


#### Finally, let's visualize the resulting clusters

In [96]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['latitude'], North_York_merged['longitude'], North_York_merged['Neighborhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters